In [3]:
import json
import io
import bz2
import collections as coll
import nltk
from nltk.corpus import cmudict
import string
import math
import numpy as np
import pandas as pd
from string import maketrans
import regex as re
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('ks772', 'x27zf4qmhv')
import itertools as itr
import glob 
from nltk.stem.snowball import SnowballStemmer

SYLLABLE_DICT = cmudict.dict()  

In [4]:
# lower
def lower_list(word_list):
    new_list = []
    for word in word_list:
        new_list.append(word.lower())
    return new_list

In [5]:
## this only takes a string...
def remove_punct(text):
    #pat = re.compile(r"[\p{P}\p{S}]+")
    pat = re.compile(r"[\p{P}\p{S}](?<!-)+") #in python 3 its r not ur: (ur"[\p{P}\p{S}]+")
    return pat.sub("", text)

# strip punc
def strip_list(word_list):
    new_list = []
    for word in word_list:
        #word_np = remove_punct_word(word)
        new_list.append(word.strip())
    return new_list


In [6]:
#stem
def stem_remover(word_list):
    return word_list
#     new_list = []
#     stemmer = SnowballStemmer("english")
#     for word in word_list:
#         new_word = stemmer.stem(word)
#         new_list.append(new_word)
#     return new_list

In [7]:
# clean
def clean_list(word_list):
    new = lower_list(word_list)
    new_new = strip_list(word_list)
    return new_new
    

In [8]:
#is three syllbs
def is_three_plus_syllables(word):
    x = SYLLABLE_DICT.get(word.lower())
    if x is None:
        return False
    return len([y for y in x[0] if y[-1].isdigit()]) >= 3

In [9]:
#is poly sylb
def count_poly_syllables(sentence):
    words = sentence.split()
    return sum(is_three_plus_syllables(word) for word in words)



In [10]:
# count sentences
def count_sents(sentence):
    return len([''.join(y) for v,y in itr.groupby(sentence, lambda x: x not in '.!?') if v])

In [11]:
#compute smog
def compute_smog_score(list_tweets):
    grade = 0
    tot_num_pollys = 0
    total_num_sentences = 0

    for tweet in list_tweets:
        total_num_sentences += count_sents(tweet)
        tot_num_pollys += count_poly_syllables(tweet)

    if total_num_sentences <30: 
        print "too few tweets"
        return None

    else:
        grade = 1.0430*(math.sqrt(tot_num_pollys*(30.0/total_num_sentences))) + 3.1291
        return grade

In [12]:
#create sat list
def create_sat_list():
    sat_word_list = []
    with open('/Users/sports195admin/Desktop/rxch/github/research/SATGRE/SATWords.txt', 'r') as infile:
        for line in infile:
            sat_word_list.append(line.strip("\n").split()[0]) # add first word of every line to list
        clean_sat = clean_list(sat_word_list)
        return clean_sat

In [13]:
#create gre list
def create_gre_list():
    gre_word_list = []
    with open('/Users/sports195admin/Desktop/rxch/github/research/SATGRE/GREwords.txt', 'r') as infile:
        for line in infile:
            gre_word_list.append(line.strip("\n").split()[0]) # add first word of every line to list
        clean_gre = clean_list(gre_word_list)
        return clean_gre

In [19]:
# word match counter
# word_set_dict - dictionary with two keys ("sat" and "gre') - "sat" - sat word set

def word_match_counter(clean_words, word_set_dict):
    result = {k:0 for k in word_set_dict}
    total_count = len(clean_words)
    word_counter = coll.Counter(clean_words)
    for word, count in word_counter.items():
        for k, v in word_set_dict.items():
            if word in v:
                #print(word)
                result[k] += count
    #print(result)            
    return {k:v/float(total_count) for k,v in result.items()}

In [31]:
# open files/texts/tweets

def main():
    #print("loading words")
    sat_words = set(create_sat_list())
    gre_words = set(create_gre_list())

    dict_of_word_sets = {"sat":sat_words, "gre" : gre_words}
    #print("done loading word list")

    in_file_name = "/Users/sports195admin/Downloads/success_timelines_00000.json.bz2"
    out_file_name = "/Users/sports195admin/Downloads/user_scores_00000.json.bz2"

    with bz2.BZ2File(in_file_name) as infile, bz2.BZ2File(out_file_name, 'w') as outfile:
        list_of_sat_nums = []   
        list_of_sat_percents = []
        list_of_gre_nums = []
        list_of_gre_percents = []
        satlist = []
        grelist = []
        
        #print("beginning loading in text")
        for line in infile: 

            first_tab = line.index("\t") # location of first tab char
            first_tab
            id = line[:first_tab]

            timeline = json.loads(line[first_tab:]) # creates a dict

            user_id = timeline[u'user'][0]["id_str"] # gets user id as string

            list_tweets = [x['text'] for x in timeline[u'tweets']] # makes all the user's tweets into a list      
            #print("got tweet list")
            smog_score = compute_smog_score(list_tweets) #computes the user's smog score and stores it in this var
            #print("finished smog score")
            usable_sample = ""
        
            for tweet in list_tweets:
                usable_sample += remove_punct(tweet.lower()) # removes all punct and makes evrything lower
            #print("merged clean tweets")
            list_words = usable_sample.split()
            
            stripped_list_words = strip_list(list_words)
            
            clean_words = stem_remover(stripped_list_words)
            
            word_count_scores = word_match_counter(clean_words, dict_of_word_sets)
            
            tdict = coll.OrderedDict() 
            tdict["uid"] = user_id
            tdict["smog"] = smog_score
            tdict["sat"] = word_count_scores["sat"]
            tdict["gre"] = word_count_scores["gre"]
            satlist.append(tdict["sat"])
            grelist.append(tdict["gre"])

            outfile.write(json.dumps(tdict)+'\n')

            #print(tdict)
            
 #   print satlist
    print grelist
    #x = tdict["sat"]

#     data = [
#         go.Histogram(
#             x=tdict["sat"]
#         )
#     ]
#     print(py.iplot(data))
#             #break
        

In [32]:
main()

[0.003012854847348688, 0.004469273743016759, 0.0015849430411094601, 0.002814720990781789, 0.002663238130223161, 0.002557148392096087, 0.0027851920622026225, 0.0035549979088247597, 0.005012531328320802, 0.003472222222222222, 0.0020541915289054094, 0.003072075620322962, 0.0032267702420077683, 0.0012091898428053204, 0.0025115110925073253, 0.002751196172248804, 0.004697712418300653, 0.005098980203959208, 0.0025693730729701952, 0.0028782894736842104, 0.0014742323013707224, 0.002427426595284807, 0.0030037246185269733, 0.002898036432458008, 0.002364998029168309, 0.0026615323631009684, 0.003225395747570343, 0.0030967632460866244, 0.0026519787841697267, 0.003444316877152698, 0.0013878374968458239, 0.0038295502019217377, 0.0032684824902723736, 0.003421240199572345, 0.002842650911320439, 0.0025821513211418063, 0.0030244530244530245, 0.003032634771194496, 0.0023642059223358356, 0.002387152777777778, 0.00356102298478472, 0.0030797998130121544, 0.003220691705480131, 0.0026013143483023, 0.00341374601

In [15]:
#create dt

#create histogram

In [16]:
# X clean in sat and gre word list maker
# X open text tweet file(s)


        
# 1 compute smog

# 2 clean tweet text

# 3 match sat words and gre words using word match counter function somehow 




# fix strip func

#  make data table func
#  make histogram func


In [17]:
# lowerX
# stripX
# stemX
# cleanX
#     lower
#     strip
#     stem
# isthreesyllbsX
# countpollyX
# countsyllbsX
# count sentsX
# compute smog scoreX

# create sat list
#     clean
# create gre list
#     clean
# open files 
#     compute smog score
#     clean
# word match counter
#dt
#histogram

In [18]:
print("loading words")
sat_words = set(create_sat_list())
gre_words = set(create_gre_list())

dict_of_word_sets = {"sat":sat_words, "gre" : gre_words}
print("done loading word list")

with open("/Users/sports195admin/Desktop/nytarticle.txt") as infile:
    y = infile.readlines()

user_id = 0
    
list_tweets = y # makes all the user's tweets into a list      
print("got tweet list")
smog_score = compute_smog_score(list_tweets) #computes the user's smog score and stores it in this var
print("finished smog score")
usable_sample = ""

for tweet in list_tweets[:10]:
    usable_sample += remove_punct(tweet.lower()) # removes all punct and makes evrything lower
print("merged clean tweets")
list_words = usable_sample.split()

stripped_list_words = strip_list(list_words)

clean_words = stem_remover(stripped_list_words)

word_count_scores = word_match_counter(clean_words, dict_of_word_sets)

tdict = coll.OrderedDict() 
tdict["uid"] = user_id
tdict["smog"] = smog_score
tdict["sat"] = word_count_scores["sat"]
tdict["gre"] = word_count_scores["gre"]

#outfile.write(json.dumps(tdict)+'\n')

print(tdict)


loading words
done loading word list


IOError: [Errno 2] No such file or directory: '/Users/sports195admin/Desktop/nytarticle.txt'

In [ ]:
print len(sat_words.intersection(gre_words))
print len(sat_words)
print len(gre_words)

In [ ]:
/Users/sports195admin/Desktop/ppr.txt

In [ ]:
print("loading words")
sat_words = set(create_sat_list())
gre_words = set(create_gre_list())
gre_not_sat = gre_words - sat_words

dict_of_word_sets = {"sat":sat_words, "gre" : gre_words, "gre only" : gre_not_sat}
print("done loading word list")

with open("/Users/sports195admin/Desktop/ppr.txt") as infile:
    y = infile.readlines()

user_id = 0
    
list_tweets = y # makes all the user's tweets into a list      
print("got tweet list")
smog_score = compute_smog_score(list_tweets) #computes the user's smog score and stores it in this var
print("finished smog score")
usable_sample = ""

for tweet in list_tweets[:10]:
    usable_sample += remove_punct(tweet.lower()) # removes all punct and makes evrything lower
print("merged clean tweets")
list_words = usable_sample.split()

stripped_list_words = strip_list(list_words)

clean_words = stem_remover(stripped_list_words)

word_count_scores = word_match_counter(clean_words, dict_of_word_sets)

tdict = coll.OrderedDict() 
tdict["uid"] = user_id
tdict["smog"] = smog_score
tdict["sat"] = word_count_scores["sat"]
tdict["gre"] = word_count_scores["gre"]
tdict["gre only"] = word_count_scores["gre only"]

#outfile.write(json.dumps(tdict)+'\n')

print(tdict)



In [30]:

list_of_sat_percents = [0.009841992501339047, 0.014972067039106146, 0.010896483407627538, 0.009921891492505806, 0.01483148131141519, 0.011235955056179775, 0.018335847742833932, 0.010769552488498537, 0.011278195488721804, 0.015625, 0.015895529687958525, 0.011185506104765656, 0.01637287122796534, 0.008779769728195153, 0.01077856843867727, 0.015191387559808613, 0.012918709150326797, 0.014797040591881624, 0.011947584789311409, 0.012266995614035088, 0.016279288604497975, 0.016825724071426195, 0.013997356722335695, 0.013721315353678732, 0.009607804493496255, 0.013987201993317855, 0.01298646182574375, 0.011241674797437746, 0.009383924928600572, 0.008897818599311137, 0.011102699974766591, 0.015735969920623868, 0.01198443579766537, 0.009265858873841768, 0.01148208015160805, 0.01701390117081108, 0.01611969111969112, 0.01267369755126058, 0.011289083279153614, 0.020833333333333332, 0.012949174490126255, 0.01545399549029313, 0.010851253592309979, 0.008762322015334063, 0.00978607191624943, 0.01005118659841787, 0.010671482515186341, 0.015855836354781102, 0.009359662139025225, 0.011204481792717087, 0.010652317153525149, 0.011046370478271426, 0.0064773293472845045, 0.009108444063437634, 0.015705484294515706, 0.016339449933701185, 0.01562270434399334, 0.013396156688839615, 0.014972075890204778, 0.01026211671612265, 0.01011764705882353, 0.010232015437777678, 0.019621342512908778, 0.013018506700701978, 0.009438084418421743, 0.011273379451703817, 0.026774958224847472, 0.011393964278382263, 0.0188500626479187, 0.013188379967920157, 0.007784998646087192, 0.010687812446129978, 0.012925463162429988, 0.014973711882229233, 0.012590603251609923, 0.010783927516249753, 0.012794136556512794, 0.016167211096908583, 0.014282313734825042, 0.013325377883850438, 0.01592495636998255, 0.010572414403255773, 0.012633996937212864, 0.009290540540540541, 0.003985932004689332, 0.012526798099962167, 0.012475015507615962, 0.011174319382364892, 0.012477255003899143, 0.0125310453827049, 0.0159870250231696, 0.01131189778703279, 0.00706090026478376, 0.013286047004022868, 0.014056224899598393, 0.016098526500989665, 0.00814503415659485, 0.009364218827008379, 0.01058987459359034, 0.011968423733129615, 0.006133258990800111, 0.0186219739292365, 0.014819064008795832, 0.018705774682470592, 0.012631443959989741, 0.009748892171344165, 0.009886519944979367, 0.013510347520499805, 0.014121750872060313, 0.009237875288683603, 0.012919896640826873, 0.010343230587145577, 0.016648820420710857, 0.01057478243270593, 0.01375831231368952, 0.011478809475129246, 0.011994329953113074, 0.010194106968300518, 0.00843795019551348, 0.01892318089659833, 0.011749031007751938, 0.010197882312098594, 0.013790386130811664, 0.015467171717171718, 0.010176125244618396, 0.01096680016468915, 0.011180763473053893, 0.010914108890189848, 0.012561441835062807, 0.015667715100385077, 0.011020172910662825, 0.013269904857285929, 0.010521393500116905, 0.01566548530243917, 0.010957919791619886, 0.011018899504846922, 0.013081021495498805, 0.009637753406447324, 0.010447761194029851, 0.009694814134924278, 0.01728665207877462, 0.010169681986740084, 0.014997794441993825, 0.013233601841196778, 0.014614343707713126, 0.0186458736923673, 0.009723546234509056, 0.01576613565445886, 0.01157930798218568, 0.011712838060143449, 0.011483202478235326, 0.01272432100390505, 0.011153601019757807, 0.01568846061957474, 0.014951760367732485, 0.00840372164815847, 0.010027302060064532, 0.019238139035793668, 0.010801652017367363, 0.012693811217990951, 0.009791971396066959, 0.011707956727391935, 0.011257999719717849, 0.009440211990725405, 0.01190784364483562, 0.018018018018018018, 0.0137574737330934, 0.007293483431108292, 0.013240161824200073, 0.015623143637875727, 0.009242834469237233, 0.010845070422535212, 0.009482667790538404, 0.012119833636599948, 0.012481810832659661, 0.011084961970053549, 0.01375238237602302, 0.011390284757118929, 0.015410317711783852, 0.010682288077188146, 0.017067410905968226, 0.011294526498696786, 0.014176961298263208, 0.017029895025102694, 0.012027353177795656, 0.010550786838340487, 0.012401950392198432, 0.010230594348814551, 0.011391625615763547, 0.014021312394840156, 0.008311848231808212, 0.01366869918699187, 0.011163779736853764, 0.01149142751797456, 0.013739409205404167, 0.012251703639263449, 0.010493139101356805, 0.012313104661389622, 0.011993011945795364, 0.008140221886693362, 0.009942703067071115, 0.011138556182131705, 0.01506276150627615, 0.014441758712480494, 0.009536784741144414, 0.011615183907078528, 0.010507246376811594, 0.012065650514113745, 0.01394573839968124, 0.0126250595521677, 0.016015537041206937, 0.0110573600552868, 0.01298502121568427, 0.015326474910770523, 0.010734312416555408, 0.01906916612798966, 0.015523483608589992, 0.012455089820359281, 0.009830558711653411, 0.013131551054057238, 0.008710517746662325, 0.01099009900990099, 0.011634377054985077, 0.01061674931899141, 0.01326807632011624, 0.008333333333333333, 0.00993140897399257, 0.00938210071098732, 0.01549053356282272, 0.010309930423782416, 0.008036196650459211, 0.010106659209843008, 0.008007626310772164, 0.012941335617982594, 0.00962320412035782, 0.012205567451820129, 0.012640571876906983, 0.010383667722632876, 0.012036728674056805, 0.01068009518012285, 0.020207876102530126, 0.011875309252845126, 0.012834224598930482, 0.012451117004539937, 0.014299695519791213, 0.011264367816091954, 0.0106752520545624, 0.012888713256527951, 0.007615970459266097, 0.009346866219393313, 0.009420608190559867, 0.015131120909723834, 0.01392972477948619, 0.013502565487442613, 0.010385978928446598, 0.009261515963928832, 0.016125269726482767, 0.009772354968084505, 0.007902571041948579, 0.01257275033577078, 0.011688476686570956, 0.0130283381364073, 0.0106951871657754, 0.011998400213304892, 0.005901961855839117, 0.008882966910948257, 0.011490549828178695, 0.009077679994456378, 0.0127506195088984, 0.013564431047475508, 0.011711281070745698, 0.009319104163535248, 0.011206419479800774, 0.014306938865349695, 0.010209905510812353, 0.01035711326539067, 0.01128133704735376, 0.0125771953607471, 0.012392494136043784, 0.014268727705112961, 0.011607257753184918, 0.014954337899543379, 0.019817130061578654, 0.014215210697262412, 0.012816991668955415, 0.009962640099626401, 0.017589820359281437, 0.01569758912416255, 0.010066824746712654, 0.009077019861597915, 0.014790549930514195, 0.015497323189630883, 0.01082277702619604, 0.009222965183306432, 0.010400550111741447, 0.013292127737347555, 0.011278195488721804, 0.015107531916221868, 0.009806983571740896, 0.011872407380918324, 0.010723065186064253, 0.013227115586953254, 0.011143575456954543, 0.012668463611859838, 0.005839065749480083, 0.01002004008016032, 0.007761053621825023, 0.013486681901622149, 0.012728068895051818, 0.012177903282739145, 0.009124224067010694, 0.010429778816759576, 0.010296191819464035, 0.010630942091616248, 0.012368300503893724, 0.008214832020848677, 0.01311267860372581, 0.013916015625, 0.013892690938954283, 0.010882937446768241, 0.014599145618182572, 0.011676396997497914, 0.010282776349614395, 0.014405144694533762, 0.010544043347733763, 0.008853945318033716, 0.013377150589599845, 0.010475774770842427, 0.01365865047863647, 0.009961910342806914, 0.01207007049775689, 0.011112967914438502, 0.010512620348685922, 0.01087381169423102, 0.010275380189066995, 0.013023337120881042, 0.026650931799183753]

x = list_of_sat_percents

data = [
    go.Histogram(
        x=list_of_sat_percents
    )
]
py.iplot(data)

In [34]:

list_of_gre_percents = [0.003012854847348688, 0.004469273743016759, 0.0015849430411094601, 0.002814720990781789, 0.002663238130223161, 0.002557148392096087, 0.0027851920622026225, 0.0035549979088247597, 0.005012531328320802, 0.003472222222222222, 0.0020541915289054094, 0.003072075620322962, 0.0032267702420077683, 0.0012091898428053204, 0.0025115110925073253, 0.002751196172248804, 0.004697712418300653, 0.005098980203959208, 0.0025693730729701952, 0.0028782894736842104, 0.0014742323013707224, 0.002427426595284807, 0.0030037246185269733, 0.002898036432458008, 0.002364998029168309, 0.0026615323631009684, 0.003225395747570343, 0.0030967632460866244, 0.0026519787841697267, 0.003444316877152698, 0.0013878374968458239, 0.0038295502019217377, 0.0032684824902723736, 0.003421240199572345, 0.002842650911320439, 0.0025821513211418063, 0.0030244530244530245, 0.003032634771194496, 0.0023642059223358356, 0.002387152777777778, 0.00356102298478472, 0.0030797998130121544, 0.003220691705480131, 0.0026013143483023, 0.0034137460172963133, 0.001892353032418179, 0.003283533081595797, 0.002164619297581038, 0.0021687022029448695, 0.004033613445378151, 0.002306171754886888, 0.002791939791211459, 0.0027404085700049826, 0.0028396913844834976, 0.0024744975255024744, 0.003250780615081911, 0.002742543709290367, 0.004600886917960088, 0.001742781320552937, 0.0032558522914606, 0.001176470588235294, 0.0023784948166763902, 0.0018932874354561102, 0.003063178047223995, 0.0034955868216376825, 0.0012810658467845247, 0.0022150545991528386, 0.0023609115171422707, 0.0027843519420854795, 0.002970355848630666, 0.002504738694828053, 0.0020111475033040283, 0.0033390779836277467, 0.0024395373291272344, 0.0016460192324352421, 0.0032991924364782353, 0.0011894046547511895, 0.002414429768498793, 0.007656907085614536, 0.0065632458233890216, 0.002399650959860384, 0.002388746350526409, 0.003445635528330781, 0.0033783783783783786, 0.0015240328253223916, 0.0028584639959645212, 0.002619064029223241, 0.0050792360828931325, 0.001819599688068625, 0.0032174305712350418, 0.0018535681186283596, 0.003335333356863022, 0.00176522506619594, 0.004763921236502223, 0.0033467202141900937, 0.002463162524741588, 0.002627430373095113, 0.0016017742730409068, 0.002415234556432884, 0.004329004329004329, 0.002509060496236409, 0.0022346368715083797, 0.0018165304268846503, 0.003080425902363892, 0.002436522185175686, 0.002481536189069424, 0.0020632737276478678, 0.0029675907848496683, 0.0020254304039608417, 0.0027348972894129086, 0.0022968705139247776, 0.004374002984613801, 0.0019343027874684812, 0.001973284760170006, 0.005044714515019491, 0.0031836901597686713, 0.0025079053538327335, 0.004049713727133082, 0.002675447622967689, 0.003003679507396561, 0.0026041666666666665, 0.0032980385349765666, 0.0026792750197005516, 0.004313973063973064, 0.002609262883235486, 0.003443500393008197, 0.0032279191616766466, 0.0018399264029438822, 0.0019115237575095577, 0.00452688203650901, 0.0033429394812680116, 0.00400600901352028, 0.002182214948172395, 0.0034249635166929745, 0.003098756006646607, 0.002022456238231397, 0.0036376997979055666, 0.003323363243602526, 0.0021616057642820383, 0.0031551170261587885, 0.00350109409190372, 0.0023598157096302954, 0.0027937068078223793, 0.002465888541837909, 0.002435723951285521, 0.003148004649360713, 0.002796313949793454, 0.0023813434061422238, 0.0042480301473107225, 0.0028295058235177995, 0.0033648453773433744, 0.0021060945109911806, 0.0027618440620352665, 0.0037031138010671337, 0.002828711420922362, 0.0021438065428975688, 0.002879126334077935, 0.004062236529470377, 0.002329768082177274, 0.003601704221021654, 0.0023159434422233057, 0.0019201049032922775, 0.0026159667398514505, 0.002484266313348791, 0.0036611072075736846, 0.002021533728850802, 0.002728275381668311, 0.0019026478515934676, 0.0027951452739977934, 0.0045740762742069625, 0.0021940061618896463, 0.002300469483568075, 0.003213570751238015, 0.001949571094359241, 0.004947453516572353, 0.0043657696374364745, 0.003251242572592399, 0.003182579564489112, 0.0024771725667540397, 0.0033310360670801746, 0.0036496350364963502, 0.003007418298469558, 0.002507179650818252, 0.003508671839342766, 0.0030571198712791635, 0.0014902241297091083, 0.001377994488022048, 0.0025982461838259174, 0.002206486042692939, 0.0026173116470368293, 0.0023088467310578365, 0.001829268292682927, 0.0031896513533867894, 0.003134025686720334, 0.004007327684909549, 0.0026098303610265334, 0.0027674213014567397, 0.003259351233897253, 0.0028802115302894375, 0.002691524978664741, 0.003033367037411527, 0.0028894999300927435, 0.0027679433537174123, 0.0026925312853777195, 0.0006811989100817438, 0.0024408719804730242, 0.002989130434782609, 0.0026750633934293755, 0.003875828594197124, 0.004192472606002858, 0.003208736770997523, 0.004549642939414881, 0.0027094729308317573, 0.00139967807404297, 0.0014953271028037382, 0.002077754178594515, 0.004017259336408272, 0.0026347305389221557, 0.0028157881736896703, 0.004122011541632316, 0.0039075219798111365, 0.002376237623762376, 0.0030856391319743033, 0.0035621987846615913, 0.002523611058004818, 0.002932098765432099, 0.0019291226064589882, 0.002565418163160595, 0.003393164494713548, 0.003668564199873498, 0.0024311183144246355, 0.0039947269604122555, 0.0036224976167778837, 0.0035171574153402174, 0.002304147465437788, 0.0014989293361884369, 0.005579286897393427, 0.0020239352340725098, 0.0030448641705123937, 0.002711526755575231, 0.001697792869269949, 0.002474022761009401, 0.0029029793735676087, 0.003326291185328359, 0.002446715963462375, 0.002528735632183908, 0.0022028297890366857, 0.0033133720400036477, 0.002654050311562428, 0.0028671368771145136, 0.002050080538878313, 0.0038988164307263867, 0.0026991854243987082, 0.002565487442614097, 0.002396764368103061, 0.0019497928345113332, 0.004024027526681052, 0.0035022312602383775, 0.0016238159675236806, 0.00212654827637666, 0.002032778554186253, 0.002761767531219981, 0.0030468847158313644, 0.0030662578322890282, 0.001256899284113886, 0.0030350136945739878, 0.0023625429553264603, 0.004157716027995288, 0.002432980400991214, 0.003265511178095956, 0.003021715378311937, 0.0030061626333984666, 0.0036317100166021027, 0.0037968414681120344, 0.0030439469845900185, 0.0036871323224790787, 0.0025069637883008357, 0.0035396897123060704, 0.0024237685691946833, 0.002887718702225242, 0.0027023549092780852, 0.004756468797564688, 0.0022765441313677926, 0.0024465347787573292, 0.00347889773871647, 0.0026567040265670404, 0.0028336184773310523, 0.0032275416890801506, 0.0024143134296184523, 0.0016176867080075492, 0.002630534048044471, 0.002704987320371936, 0.002890173410404624, 0.0018445930366612867, 0.004383702939659618, 0.0012959824543913867, 0.002699055330634278, 0.0024658988045072883, 0.0035604972203135735, 0.0030992228102798836, 0.0025006357548529286, 0.0033067788967383136, 0.0021743561867228374, 0.0037735849056603774, 0.002399616061430171, 0.0037722503831191796, 0.0021166509877704608, 0.0026973363803244298, 0.0027149321266968325, 0.00330921284857042, 0.001869718046518585, 0.0026973565905412697, 0.00232722143864598, 0.0025064822817631805, 0.0009161704076958314, 0.0014163503484221857, 0.0037981551817688553, 0.00302734375, 0.003216534355324391, 0.002365855966688748, 0.0027373398034092325, 0.0026303971258099697, 0.00317954268705182, 0.003987138263665595, 0.004246906348392766, 0.002833262501770789, 0.0010052194084670405, 0.00392841553906591, 0.003910810179780528, 0.0032229709932610606, 0.0033112582781456954, 0.0019217914438502675, 0.0021337496747332813, 0.0031905195989061076, 0.005137690094533498, 0.0036710077790402936, 0.0036386881054236123]

x = list_of_gre_percents

data = [
    go.Histogram(
        x=list_of_gre_percents
    )
]
py.iplot(data)